In [1]:
import pandas as pd
from data_path import data_path
dataPath = data_path.dataPath

data = pd.read_parquet(dataPath + 'train_series.parquet')
labels = pd.read_csv(dataPath + 'train_events.csv')

print(data)
print(labels)

              series_id    step                 timestamp     anglez    enmo
0          038441c925bb       0  2018-08-14T15:30:00-0400   2.636700  0.0217
1          038441c925bb       1  2018-08-14T15:30:05-0400   2.636800  0.0215
2          038441c925bb       2  2018-08-14T15:30:10-0400   2.637000  0.0216
3          038441c925bb       3  2018-08-14T15:30:15-0400   2.636800  0.0213
4          038441c925bb       4  2018-08-14T15:30:20-0400   2.636800  0.0215
...                 ...     ...                       ...        ...     ...
127946335  fe90110788d2  592375  2017-09-08T00:14:35-0400 -27.277500  0.0204
127946336  fe90110788d2  592376  2017-09-08T00:14:40-0400 -27.032499  0.0233
127946337  fe90110788d2  592377  2017-09-08T00:14:45-0400 -26.841200  0.0202
127946338  fe90110788d2  592378  2017-09-08T00:14:50-0400 -26.723900  0.0199
127946339  fe90110788d2  592379  2017-09-08T00:14:55-0400 -31.521601  0.0205

[127946340 rows x 5 columns]
          series_id  night   event      step  

In [2]:
def getTimeFromTimestamp(timestamp):
    sum = 0
    sum += int(timestamp[:2]) * 3600
    sum += int(timestamp[3:5]) * 60
    sum += int(timestamp[6:8])
    return sum

def getTimestampFromTime(time):
    hour = time // 3600
    minute = (time - hour * 3600) // 60
    second = time - hour * 3600 - minute * 60
    return str(hour).zfill(2) + ':' + str(minute).zfill(2) + ':' + str(second).zfill(2)

In [3]:
def getStartStep(time):
    time415 = getTimeFromTimestamp("04:15:00")  
    timeStart = getTimeFromTimestamp(time[11:-5])

    if timeStart < time415:
        return (time415 - timeStart) // 5
    else:
        return ((getTimeFromTimestamp("24:00:00") - timeStart) + time415) // 5

In [13]:
#creats a list of shape(batch_size, context_length, 2)

stepsPerNight = 24 * 3600 // 5
contextLength = getTimeFromTimestamp("08:00:00") // 5
array = []
labelsArray = []

for person in data['series_id'].unique():
    data_person = data[data['series_id'] == person]
    data_person = data_person.reset_index()

    label_person = labels[labels['series_id'] == person]
    label_person = label_person.reset_index()


    startStep = getStartStep(data_person["timestamp"][0])
    numDays = len(data_person) // stepsPerNight


    for day in range(numDays - 1):
        nightLabel = label_person[(label_person['step'] > startStep + (day * stepsPerNight)) & (label_person['step'] < startStep + contextLength + (day * stepsPerNight))]
        dayData = data[startStep + (day * stepsPerNight) : startStep + contextLength + (day * stepsPerNight)]

        array.append([dayData["enmo"], dayData["anglez"]])

        try:
            print(nightLabel["step"] - (startStep + stepsPerNight * day))
            labelsArray.append(float(nightLabel["step"] - (startStep + stepsPerNight * day)))
            print("pass")
        except:
            labelsArray.append(5759.0)


    #only for debugging
    break
    


1    1752.0
Name: step, dtype: float64
pass
3    1032.0
Name: step, dtype: float64
pass
5    660.0
Name: step, dtype: float64
pass
7    1836.0
Name: step, dtype: float64
pass
Series([], Name: step, dtype: float64)
11    2280.0
Name: step, dtype: float64
pass
13    5664.0
Name: step, dtype: float64
pass
15    3192.0
Name: step, dtype: float64
pass
Series([], Name: step, dtype: float64)
19    2700.0
Name: step, dtype: float64
pass
Series([], Name: step, dtype: float64)
23    3012.0
Name: step, dtype: float64
pass
25    2844.0
Name: step, dtype: float64
pass
Series([], Name: step, dtype: float64)
29    2604.0
Name: step, dtype: float64
pass
31    1752.0
Name: step, dtype: float64
pass
33    2112.0
Name: step, dtype: float64
pass
35    2292.0
Name: step, dtype: float64
pass
37    2376.0
Name: step, dtype: float64
pass
39    1560.0
Name: step, dtype: float64
pass
41    1164.0
Name: step, dtype: float64
pass


In [11]:
import numpy as np

numpyData = np.array(array)
numpyLabels = np.array(labelsArray)

np.save("batched_data\\features", numpyData)
np.save("batched_data\\labels", numpyLabels)
